<a href="https://colab.research.google.com/github/VFA21SCM91V/ML_projects/blob/main/Dutch%20to%20EN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov 10 19:18:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install sentencepiece
!pip install plotly

     |████████████████████████████████| 1.2 MB 7.1 MB/s 


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import sentencepiece as sp
import time
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from numpy.random import shuffle

init_notebook_mode(connected=True)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving nld.csv to nld.csv


In [ ]:
import io
train_df = pd.read_csv('/content/nld.csv')

In [ ]:
train_df.head()

,Unnamed: 0,Hi.,Hoi.
0,0,Run!,Ren!
1,1,Who?,Wie?
2,2,Wow!,Da's niet gek!
3,3,Fire!,Vuur!
4,4,Fire!,Brand!


In [ ]:
train_df

,Unnamed: 0,Hi.,Hoi.
0,0,Run!,Ren!
1,1,Who?,Wie?
2,2,Wow!,Da's niet gek!
3,3,Fire!,Vuur!
4,4,Fire!,Brand!
...,...,...,...
41136,41136,The world's first Ferris wheel was built in Ch...,Het eerste reuzenrad ter wereld werd gebouwd i...
41137,41137,"Jingle Bells, a popular song around Christmas ...","„Jingle Bells”, het bekende kerstlied, heeft i..."
41138,41138,Mary thought to herself that she ought to buy ...,Mary dacht bij zichzelf dat ze een nieuwe bede...
41139,41139,Always use distilled water in steam irons beca...,Gebruik altijd gedistilleerd water in stoomstr...


In [ ]:
del train_df['Unnamed: 0']

In [ ]:
train_df.columns = ['EN', "NL"]

In [ ]:
train_df.head()

,EN,NL
0,Run!,Ren!
1,Who?,Wie?
2,Wow!,Da's niet gek!
3,Fire!,Vuur!
4,Fire!,Brand!


In [ ]:
def write_train_dfner_file(col, filename):
    texts = list(col.values)
    with open(filename, 'w',encoding='utf-8') as f:
        for text in texts:
            f.write(text + "\n")

In [ ]:
en_sp_train_dfner = "en_spm.txt"
write_train_dfner_file(train_df["EN"], en_sp_train_dfner)

In [ ]:
nl_sp_train_dfner = "nl_spm.txt"
write_train_dfner_file(train_df["NL"], nl_sp_train_dfner)

In [ ]:
def createSPModel(train_dfner_file, model_prefix, vocab_size, sp):
    spm_train_dfn_param = f"--input={train_dfner_file} --model_prefix={model_prefix} --vocab_size={vocab_size}"
    sp.SentencePieceTrainer.Train(spm_train_dfn_param)
    lang_sp = sp.SentencePieceProcessor()
    lang_sp.Load(f"{model_prefix}.model")
    return lang_sp
    

In [ ]:
en_sp = createSPModel(en_sp_train_dfner, "en_sp", 7100, sp)

In [ ]:
print(en_sp.EncodeAsPieces("This is a test."))
print(en_sp.EncodeAsIds("This is a test."))
print(en_sp.DecodeIds(en_sp.EncodeAsIds("This is a test.")))

['▁Thi', 's', '▁is', '▁a', '▁test', '.']
[69, 9, 13, 11, 1263, 3]
This is a test.


In [ ]:
nl_sp = createSPModel(nl_sp_train_dfner, "nl_sp", 9600, sp)

In [ ]:
nl_sp.EncodeAsPieces("Ik wil graag een fles water.")

['▁I', 'k', '▁wil', '▁graag', '▁een', '▁fles', '▁water', '.']

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
import keras as K

In [ ]:
def encode_sentence(df, lang, spm):
    lang_pieces = []
    lang_lens = []
    for index, row in df.iterrows():
        lang_piece = spm.EncodeAsIds(row[lang])
        lang_pieces.append(lang_piece)
        lang_lens.append(len(lang_piece)) 
    df[f"{lang}_pieces"] = lang_pieces
    df[f"{lang}_len"] = lang_lens

In [ ]:

start_time = time.time()
encode_sentence(train_df, "EN", en_sp)
encode_sentence(train_df, "NL", nl_sp)
print(f"Encoding time: {time.time()-start_time} sec")


Encoding time: 6.409631967544556 sec


In [ ]:
train_df.tail()

,EN,NL,EN_pieces,EN_len,NL_pieces,NL_len
41136,The world's first Ferris wheel was built in Ch...,Het eerste reuzenrad ter wereld werd gebouwd i...,"[22, 429, 5, 9, 283, 4031, 3629, 20, 824, 16, ...",31,"[26, 396, 1747, 5510, 4620, 683, 451, 139, 105...",30
41137,"Jingle Bells, a popular song around Christmas ...","„Jingle Bells”, het bekende kerstlied, heeft i...","[1747, 6942, 3785, 9, 24, 11, 1065, 680, 389, ...",26,"[15, 0, 4578, 9079, 4344, 41, 0, 30, 11, 4446,...",39
41138,Mary thought to herself that she ought to buy ...,Mary dacht bij zichzelf dat ze een nieuwe bede...,"[45, 231, 7, 1615, 23, 158, 15, 852, 7, 200, 1...",28,"[100, 242, 115, 860, 14, 89, 9, 195, 7146, 411...",29
41139,Always use distilled water in steam irons beca...,Gebruik altijd gedistilleerd water in stoomstr...,"[120, 4372, 281, 5526, 2945, 182, 16, 4200, 14...",33,"[1635, 132, 7129, 777, 6987, 169, 16, 5509, 62...",32
41140,If someone who doesn't know your background sa...,Als iemand die je achtergrond niet kent zegt d...,"[4, 194, 685, 164, 80, 19, 5, 14, 42, 29, 167,...",60,"[212, 222, 79, 12, 419, 3416, 13, 555, 264, 14...",53


In [ ]:
def plotLangLen(lang1, lang2):
    trace1 = go.Histogram(
        x=train_df[f"{lang1}_len"].values,
        opacity=0.75,
        name = f"Length of {lang1} sentences",
        marker=dict(color='rgba(171, 50, 96, 0.6)'))
    trace2 = go.Histogram(
        x=train_df[f"{lang2}_len"].values,
        opacity=0.75,
        name = f"Length of {lang2} sentences",
        marker=dict(color='rgba(12, 50, 196, 0.6)'))

    data = [trace1, trace2]
    layout = go.Layout(barmode='overlay',
                       title=f"Lengths of {lang1} and {lang2} sentences",
                       xaxis=dict(title='Length'),
                       yaxis=dict( title='Count'),
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, config={'showLink': True})

In [ ]:
plotLangLen("EN", "NL")

In [ ]:
en_vocab_size = en_sp.get_piece_size()
nl_vocab_size = nl_sp.get_piece_size()
print(f"English vocab size: {en_vocab_size}")
print(f"NL vocab size: {nl_vocab_size}")

English vocab size: 7100
NL vocab size: 9600


In [ ]:
print(en_sp.piece_to_id('__MUST_BE_UNKNOWN__'))
print(en_sp.id_to_piece(0))

0
<unk>


In [ ]:
en_max_length = train_df["EN_len"].max()
nl_max_length = train_df["NL_len"].max()
#30 for faster processing time
en_max_length=30
nl_max_length=en_max_length
print(en_max_length)
print(nl_max_length)

30
30


In [ ]:
#use post padding to fill up short sentence with 0
en_padded_seq = pad_sequences(train_df["EN_pieces"].tolist(), maxlen=en_max_length, padding='post')
nl_padded_seq = pad_sequences(train_df["NL_pieces"].tolist(), maxlen=nl_max_length, padding='post')

In [ ]:
#pick a sample
en_padded_seq[2]

array([3733,   97,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
train_seq_df = pd.DataFrame( {'en_seq':en_padded_seq.tolist(), 'nl_seq':nl_padded_seq.tolist()})

In [ ]:
train_seq_df.tail()

,en_seq,nl_seq
41136,"[429, 5, 9, 283, 4031, 3629, 20, 824, 16, 1538...","[26, 396, 1747, 5510, 4620, 683, 451, 139, 105..."
41137,"[1747, 6942, 3785, 9, 24, 11, 1065, 680, 389, ...","[4446, 1393, 7322, 30, 29, 16, 1994, 877, 152,..."
41138,"[45, 231, 7, 1615, 23, 158, 15, 852, 7, 200, 1...","[100, 242, 115, 860, 14, 89, 9, 195, 7146, 411..."
41139,"[5526, 2945, 182, 16, 4200, 1423, 9, 41, 380, ...","[7129, 777, 6987, 169, 16, 5509, 6240, 41, 30,..."
41140,"[808, 23, 181, 337, 1716, 12, 84, 19, 5, 14, 1...","[12, 247, 4834, 29, 14, 69, 220, 5418, 905, 14..."


In [ ]:
def define_model(input_vocab,output_vocab, input_length,output_length,output_dim):
        model = Sequential()
      #mark_zero , set 0 as special character reserved for unknown words  
        model.add(Embedding(input_vocab, output_dim, input_length=input_length, mask_zero=True))
        model.add(LSTM(output_dim))
      #repeat the input (n) times
#return_sequences=True returns all the outputs the encoder observed in the past, while RepeatVector repeats the very last output of the encoder.    
        model.add(RepeatVector(output_length))
    #return the full sequences
        model.add(LSTM(output_dim, return_sequences=True))
      #model.add(TimeDistributed(Dense(output_vocab, activation='softmax')))
      
        model.add(Dense(output_vocab, activation='softmax'))
        return model

In [ ]:
train_seq_df.shape

(41141, 2)

In [ ]:
train_dfn, test = train_test_split(train_seq_df, test_size=0.1, random_state = 3)

In [ ]:
train_dfnX = np.asarray(train_dfn["nl_seq"].tolist())
train_dfnY = np.asarray(train_dfn["en_seq"].tolist())

testX = np.asarray(test["nl_seq"].tolist())
testY = np.asarray(test["en_seq"].tolist())

In [ ]:
model = define_model(nl_vocab_size, en_vocab_size, nl_max_length, en_max_length, 1024)

In [ ]:
#RMSprop is recommended for RNN, sparse_categorical_crossentropy for densed target output as integers
model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='sparse_categorical_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [ ]:
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
    ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [ ]:
filename = 'nmt_model'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# train_dfn model
history = model.fit(train_dfnX, train_dfnY.reshape(train_dfnY.shape[0], train_dfnY.shape[1], 1),
                    epochs=15, batch_size=64, validation_split = 0.1,callbacks=[checkpoint], 
                    verbose=1)


Epoch 1/15
521/521 [==============================] - ETA: 0s - loss: 1.4844
Epoch 00001: val_loss improved from inf to 1.27006, saving model to nmt_model


INFO:tensorflow:Assets written to: nmt_model/assets


INFO:tensorflow:Assets written to: nmt_model/assets


521/521 [==============================] - 61s 104ms/step - loss: 1.4844 - val_loss: 1.2701
Epoch 2/15
521/521 [==============================] - ETA: 0s - loss: 1.1534
Epoch 00002: val_loss improved from 1.27006 to 1.10269, saving model to nmt_model


INFO:tensorflow:Assets written to: nmt_model/assets


...

INFO:tensorflow:Assets written to: nmt_model/assets


521/521 [==============================] - 52s 100ms/step - loss: 0.7452 - val_loss: 0.8853
Epoch 7/15
521/521 [==============================] - ETA: 0s - loss: 0.6729
Epoch 00007: val_loss improved from 0.88533 to 0.86437, saving model to nmt_model


INFO:tensorflow:Assets written to: nmt_model/assets


INFO:tensorflow:Assets written to: nmt_model/assets


521/521 [==============================] - 52s 100ms/step - loss: 0.6729 - val_loss: 0.8644
Epoch 8/15
520/521 [============================>.] - ETA: 0s - loss: 0.6055
Epoch 00008: val_loss did not improve from 0.86437
521/521 [==============================] - 39s 75ms/step - loss: 0.6055 - val_loss: 0.8781
Epoch 9/15
521/521 [==============================] - ETA: 0s - loss: 0.5448
Epoch 00009: val_loss did not improve from 0.86437
521/521 [==============================] - 38s 74ms/step - loss: 0.5448 - val_loss: 0.8729
Epoch 10/15
520/521 [============================>.] - ETA: 0s - loss: 0.4911
Epoch 00010: val_loss did not improve from 0.86437
521/521 [==============================] - 39s 76ms/step - loss: 0.4911 - val_loss: 0.8740
Epoch 11/15
520/521 [============================>.] - ETA: 0s - loss: 0.4441
Epoch 00011: val_loss did not improve from 0.86437
521/521 [==============================] - 40s 77ms/step - loss: 0.4441 - val_loss: 0.8934
Epoch 12/15
520/521 [=========

In [ ]:
trace1 = go.Scatter(
    y=history.history['loss'],
    name = "Training Loss",
    marker=dict(color='rgba(171, 50, 96, 0.6)'))
trace2 = go.Scatter(
    y=history.history['val_loss'],
    name = "Validation Loss",
    marker=dict(color='rgba(12, 50, 196, 0.6)'))

data = [trace1, trace2]
layout = go.Layout(title='Loss and Val_Loss in 15 Epochs',
                   xaxis=dict(title='Epoch'),
                   yaxis=dict( title='Loss'),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, config={'showLink': True})

In [ ]:
model = load_model('nmt_model')

In [ ]:
testX.shape[0], testX.shape[1]

(4115, 30)

In [ ]:
preds = model.predict(testX.reshape((testX.shape[0],testX.shape[1])))

In [ ]:
def get_word(ids, tokenizer):
    return tokenizer.DecodeIds(list(filter(lambda a: a != 0, ids.tolist())))


In [ ]:
import random

In [ ]:
y_indexs = random.sample(range(len(testY)),  30)

for y_index in y_indexs: 
    print(f"--- Index: {y_index}")
    print(f"NL: {get_word(testX[y_index], nl_sp)}") 
    print(f"EN: {get_word(testY[y_index], en_sp)}") 

 # print(get_word(k_preds[0][y_index], en_sp))
 # print(get_word(k_preds[1][y_index], en_sp))
 # print(get_word(k_preds[2][y_index], en_sp))


--- Index: 267
NL: Ik heb je mijn antwoord al gegeven.
EN: I've already given you my answer.
--- Index: 2617
NL: Ik ben bij het politiebureau.
EN: I'm at the police station.
--- Index: 4008
NL: Hij gleed uit over een bananenschil.
EN: He slipped on a banana peel.
--- Index: 2141
NL: Het is ons gelukt!
EN: We succeeded!
--- Index: 2099
NL: Mijn grootmoeder was een pachter.
EN: My grandmother was a farmer.
--- Index: 2686
NL: De taart is vers.
EN: The pie is fresh.
--- Index: 947
NL: Tom is stoned.
EN: Tom is stoned.
--- Index: 1104
NL: Kinderen houden van ijs.
EN: Kids like ice cream.
--- Index: 1625
NL: Ik betaal niet graag belasting.
EN: I don't like paying taxes.
--- Index: 3509
NL: Tom werd kwaad en schold Mary uit.
EN: Tom got mad and called Mary names.
--- Index: 2128
NL: Het hoost.
EN: It is raining hard.
--- Index: 2739
NL: Nou, laten we gaan.
EN: Well, let's go.
--- Index: 1328
NL: Over smaak valt niet te twisten.
EN: There's no accounting for taste.
--- Index: 854
NL: De dreum

In [ ]:
test_ids = []
test_nls = []
test_ens = []
test_mts = []
for y_index in range(len(testY)): 
    test_ids.append(y_index)
    test_nls.append(get_word(testX[y_index], nl_sp))
    test_ens.append(get_word(testY[y_index], en_sp))


predict_df = pd.DataFrame( {'id':test_ids, 'NL':test_nls, 'English':test_ens})

In [ ]:
pd.set_option('display.max_colwidth', 80)


In [ ]:
predict_df.sample(10)

,id,NL,English
1312,1312,Je bent me een biertje schuldig.,You owe me a beer.
4057,4057,Ik betwijfel het dat Tom schuldig is.,I doubt that Tom is guilty.
1914,1914,Gisteren heeft Maria die rok gekocht.,It was yesterday that Mary bought this skirt.
1075,1075,"Hoelang moeten we wachten, denk je?",How long do you think we'll have to wait?
653,653,Wees vriendelijk.,Be friendly.
2021,2021,Je kunt dat niet ontkennen.,You can't deny that.
3158,3158,Tom is je broer.,Tom is your brother.
488,488,Ik heb je lof niet nodig.,I don't need your praise.
644,644,Hoeveel vissen heb je gevangen?,How many fish did you catch?
3104,3104,Mijn kinderen mogen van mij geen televisie kijken op een doordeweekse avond.,I don't allow my kids to watch TV on school nights.


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
chinese = pd.read_csv('/content/gdrive/My Drive/ML_Project/chinese.txt', names=['Chinese'])

In [ ]:
english = pd.read_csv('/content/gdrive/My Drive/ML_Project/eng.txt', names=['English'])